In [1]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff

In [2]:
ticker = 'msft'
start = '2016-8-31'
end = '2021-8-31'
stock_data = yf.download(ticker, start, end, adjusted=True)

[*********************100%***********************]  1 of 1 completed


In [3]:
ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)

In [4]:
stock_returns = stock_data['Adj Close'].resample('M').last().pct_change().dropna()
stock_returns.name = "Month_Rtn"
ff_data = ff3_monthly.merge(stock_returns,on='Date')

In [5]:
X = ff_data[['Mkt-RF', 'SMB', 'HML']]
y = ff_data['Month_Rtn'] - ff_data['RF']
X = sm.add_constant(X)

In [6]:
ff_model = sm.OLS(y, X).fit()http://localhost:8888/notebooks/Fama-French%203%20Factor.ipynb#
print(ff_model.summary())
intercept, b1, b2, b3 = ff_model.params

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     28.98
Date:                Fri, 07 Jul 2023   Prob (F-statistic):           1.92e-11
Time:                        23:15:54   Log-Likelihood:                 126.50
No. Observations:                  60   AIC:                            -245.0
Df Residuals:                      56   BIC:                            -236.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0155      0.004      3.671      0.0

In [7]:
rf = ff_data['RF'].mean()
market_premium = ff3_monthly['Mkt-RF'].mean()
size_premium = ff3_monthly['SMB'].mean()
value_premium = ff3_monthly['HML'].mean()
expected_monthly_return = rf + b1 * market_premium + b2 * size_premium + b3 * value_premium 
expected_yearly_return = expected_monthly_return * 12
print("Expected yearly return: " + str(expected_yearly_return))

Expected yearly return: 0.059528149928772284
